# Apriori algorithm

In [1]:
from collections import defaultdict
from itertools import combinations, chain

In [2]:
support = .05

In [3]:
def apriori(data, support):
    candidates = list(map(lambda x: frozenset([x]), set(chain(*data))))
    result = dict()
    k = 2
    def scan():
        count = defaultdict(int)
        for transaction in data:
            for candidate in candidates:
                if candidate.issubset(transaction):
                    count[candidate] += 1
        return {k: v/len(data) for k, v in count.items() if float(v)/len(data) >= support}
    while candidates:
        filtered = scan()
        result[k - 1] = filtered
        candidates = set([i.union(j) for i in filtered.keys() for j in filtered.keys() if len(i.union(j)) == k])
        k += 1
    return result

In [4]:
def define(freq, transactions, confidence=.0):
    def support(item):
        return float(freq[len(item)][item])/len(transactions)
    
    for k, v in freq.items():
        if k == 1: continue
        for item in v:
            for element in map(frozenset, chain(*[combinations(item, i) for i, e in enumerate(item, 1)])):
                remain = item.difference(element)
                if remain:
                    conf = support(item) / support(element)
                    if conf >= confidence:
                        yield element, remain, round(freq[len(item)][item] * 100, 2), round(conf * 100, 2)

In [5]:
with open('input.txt') as f:
    data = [list(map(int, line.split())) for line in f.readlines()]

In [6]:
freq = apriori(data, .05)

In [7]:
rules = list(define(freq, data))

In [8]:
with open('output.txt', 'w') as f:
    for item, asso, sup, conf in rules:
        f.write('{{{}}}\t{{{}}}\t{:.2f}\t{:.2f}\n'.format(','.join(map(str, item)), ','.join(map(str, asso)), sup, conf))